# 3. Pre-Processing

In [24]:
import numpy as np
from HMM import unsupervised_HMM
from HMM_helper import (
    text_to_wordcloud,
    states_to_wordclouds,
    sample_sentence,
    visualize_sparsities,
    animate_emission
)

In [29]:
# Load the data
filename = 'data/shakespeare.txt'

with open(filename, 'r') as f:
    content = f.readlines()
    lines = []
    for line in content:
        line = line.strip(',')
        if len(line) > 23:
            words = (line.split('\n')[0].split(' '))
            while '' in words : words.remove('')
            newWords = []
            for w in words:
                w = w.strip(' ,.:;?!\'()').lower()
                newWords.append(w)
            lines.append(newWords)

In [31]:
def parse_observations(lines):
    obs_counter = 0
    obs = []
    obs_map = {}

    for line in lines:
        obs_elem = []
        
        for word in line:
            if word not in obs_map:
                # Add unique words to the observations map.
                obs_map[word] = obs_counter
                obs_counter += 1
            
            # Add the encoded word.
            obs_elem.append(obs_map[word])
        
        # Add the encoded sequence.
        obs.append(obs_elem)

    return obs, obs_map

In [32]:
obs, obs_map = parse_observations(lines)

In [33]:
hmm = unsupervised_HMM(obs, 5, 50)

Iteration: 10
Iteration: 20
Iteration: 30
Iteration: 40
Iteration: 50


In [25]:
print('Sample Sentence:\n====================')
print(sample_sentence(hmm, obs_map, n_words=25))

Sample Sentence:
That longer methinks do besides are when and give heart, numbers (no predict i made your self by swear old spurring ragged my treasure worth....
